In [1]:
import pandas as pd

In [2]:
# import PGA data
pga_df = pd.read_csv('PGA_Data.csv')

In [15]:
# drop fantasy related and filler columns
pga_df.drop(columns=['hole_DKP', 'hole_FDP', 'hole_SDP', 'streak_DKP',
       'streak_FDP', 'streak_SDP', 'n_rounds', 'made_cut', 'pos', 'finish_DKP',
       'finish_FDP', 'finish_SDP', 'total_DKP', 'total_FDP', 'total_SDP',
       'player', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',])

,Player_initial_last,tournament id,player id,hole_par,strokes,tournament name,course,date,purse,season,no_cut,Finish,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total
0,A. Hadwin,401353275,5548,284,283,BMW Championship,"Wilmington Country Club - Wilmington, DE",2022-08-21,15.0,2022,1,T44,-0.56,0.23,-0.03,-0.13,0.06,-0.50
1,A. Noren,401353275,3832,284,285,BMW Championship,"Wilmington Country Club - Wilmington, DE",2022-08-21,15.0,2022,1,T52,0.99,-0.09,-1.63,-0.28,-1.99,-1.00
2,A. Putnam,401353275,5502,284,279,BMW Championship,"Wilmington Country Club - Wilmington, DE",2022-08-21,15.0,2022,1,T23,1.30,0.49,-0.27,-1.01,-0.80,0.50
3,A. Scott,401353275,388,284,274,BMW Championship,"Wilmington Country Club - Wilmington, DE",2022-08-21,15.0,2022,1,T5,0.70,0.28,0.27,0.50,1.05,1.75
4,A. Smalley,401353275,9484,284,283,BMW Championship,"Wilmington Country Club - Wilmington, DE",2022-08-21,15.0,2022,1,T44,-0.23,-1.08,1.31,-0.49,-0.27,-0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38210,V. Singh,2271,392,144,146,Frys.com Open,"Silverado Resort - Napa, CA",2014-10-12,6.0,2015,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38211,W. Kim,2271,7082,144,150,Frys.com Open,"Silverado Resort - Napa, CA",2014-10-12,6.0,2015,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38212,W. McGirt,2271,3532,216,215,Frys.com Open,"Silverado Resort - Napa, CA",2014-10-12,6.0,2015,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38213,Z. Blair,2271,9040,288,278,Frys.com Open,"Silverado Resort - Napa, CA",2014-10-12,6.0,2015,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# make a column that is the finish, removing ties and turning 'CUT' to 0. 

# 0 could be changed to 99, NA, or whatever the cut line is (count in tournament / 2)

# note, the data column is an 'Object' type so it has strings and integers. 
# this methods turns everything to a string, makes its caclualtions, then turns to a float so we can do statistical calculations.

def adjust_finish(x):
    # this treats cut, dq, wd, etc. all as the same and assigns a value of 99
    # todo: determine if we want to treat these all the same
    # todo: because lower is better, NFs are treated as 99, but we can hone this in, 
    if x == 'CUT' or x == 'DQ' or x == 'WD' or x == 'MDF' or x == 'W/D':
        return '99'
    elif x.startswith('T'):
        return x[1:]
    else:
        return x
pga_df['finish_adj'] = pga_df['Finish'].astype(str).apply(adjust_finish).astype(float)


In [36]:
pga_df[pga_df['Player_initial_last'] == 'S. Scheffler'][['sg_total', 'Finish', 'finish_adj']]

,sg_total,Finish,finish_adj
52,2.00,T3,3.0
159,-0.51,CUT,99.0
648,1.61,T21,21.0
842,-0.22,CUT,99.0
1114,1.85,T13,13.0
...,...,...,...
19977,0.30,T45,45.0
21884,0.69,T43,43.0
23699,-2.33,CUT,99.0
26555,NaN,NaN,NaN
